In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1292975/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_39_9,0.999718,0.652973,0.592240,0.999716,0.000144,0.206010,0.285644,0.000123,0.004632,0.012006,1.000227,0.012428,147.689251,238.767080,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_39_8,0.999714,0.653199,0.592472,0.999721,0.000146,0.205876,0.285482,0.000120,0.004662,0.012088,1.000230,0.012513,147.662034,238.739863,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_40_0,0.999713,0.653746,0.593223,0.999168,0.000147,0.205551,0.284956,0.000141,0.004629,0.012114,1.000231,0.012539,147.653677,238.731507,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_39_7,0.999710,0.653432,0.592715,0.999726,0.000148,0.205738,0.285312,0.000118,0.004692,0.012173,1.000233,0.012600,147.634149,238.711979,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_40_1,0.999707,0.654517,0.594204,0.999007,0.000150,0.205094,0.284268,0.000168,0.004626,0.012241,1.000236,0.012671,147.611827,238.689656,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,model_1_0_4,0.909655,0.567770,0.415658,0.806020,0.046196,0.256590,0.409344,0.098679,0.125681,0.214934,1.072778,0.222477,136.149706,227.227536,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
495,model_1_0_3,0.908403,0.567798,0.415561,0.805013,0.046837,0.256574,0.409411,0.099191,0.126696,0.216418,1.073786,0.224014,136.122178,227.200008,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
496,model_1_0_2,0.907124,0.567802,0.415443,0.803973,0.047491,0.256571,0.409495,0.099720,0.127720,0.217925,1.074817,0.225573,136.094426,227.172256,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
497,model_1_0_1,0.905814,0.567806,0.415333,0.802896,0.048160,0.256569,0.409571,0.100268,0.128755,0.219455,1.075872,0.227157,136.066434,227.144264,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
